In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Setup logging with some hacks to work in the notebook.
import logging
import sys

logger = logging.getLogger()
logger.addHandler(logging.StreamHandler())
logger.setLevel(logging.INFO)

import matplotlib.pyplot as plt
import numpy as np
import tifffile as tiff
import keras.backend as K
from keras.metrics import binary_crossentropy
from math import sqrt
from skimage.transform import resize

import sys; sys.path.append('../')
from src.models.unet_jocic import UNet
from src.utils.runtime import gpu_selection
from src.utils.data import random_transforms
from src.utils.model import dice_coef, jaccard_coef

gpu_selection(visible_devices='4')

logger.info('Done')

Using TensorFlow backend.
Done


In [ ]:
model = UNet()

exp = 32 ###
model.checkpoint_name = 'checkpoints/{}'.format(exp)  

model.config['data_path'] = 'data'
model.config['checkpoint_path_config'] = model.checkpoint_name + '.config'
model.config['checkpoint_path_history'] = model.checkpoint_name + '.history'
model.config['batch_size'] = 1

# model.config['prop_trn'] = 20./30
# model.config['prop_val'] = 10./30
# model.config['montage_trn_shape'] = (5, 4)
# model.config['montage_val_shape'] = (5, 2)
model.config['prop_trn'] = 30./30
model.config['prop_val'] = 30./30
model.config['montage_trn_shape'] = (5, 6)
model.config['montage_val_shape'] = (6, 5)

model.config['early_stop_patience'] = 30
model.config['aug'] = False  # elastic deformation add
model.config['steps'] = 500
model.config['random_split'] = False


# load_data train/val split will be same every time
# but batch_gen will re_seed
np.random.seed(model.config['seed'])  
model.load_data()
model.save_config()  

model.compile(multi_gpu=False)
# model.net.summary()

model.train()

Reading images from data.
Combining images and masks into montages.


[27  9 21 10 13 24 26  4  5  6 20 18  1 17 15  2 12 28 29  8 23  3 16 25
  7 11 22 19 14  0]
[27  9 21 10 13 24 26  4  5  6 20 18  1 17 15  2 12 28 29  8 23  3 16 25
  7 11 22 19 14  0]


Combining validation images and masks into montages
Saving model config to checkpoints/32.config.
Training for 100 epochs.


Epoch 1/100
500/500 [==============================] - 36s 71ms/step - loss: 0.6580 - dice_coef: 0.8714 - jaccard_coef: 0.6459 - val_loss: 0.5807 - val_dice_coef: 0.8785 - val_jaccard_coef: 0.6443

Epoch 00001: val_loss improved from inf to 0.58071, saving model to checkpoints/32_val_loss.net

Epoch 00001: loss improved from inf to 0.65797, saving model to checkpoints/32_trn_loss.net
Epoch 2/100
500/500 [==============================] - 28s 56ms/step - loss: 0.5344 - dice_coef: 0.9025 - jaccard_coef: 0.6762 - val_loss: 0.4883 - val_dice_coef: 0.9030 - val_jaccard_coef: 0.6735

Epoch 00002: val_loss improved from 0.58071 to 0.48834, saving model to checkpoints/32_val_loss.net

Epoch 00002: loss improved from 0.65797 to 0.53441, saving model to checkpoints/32_trn_loss.net
Epoch 3/100
500/500 [==============================] - 28s 55ms/step - loss: 0.4937 - dice_coef: 0.9147 - jaccard_coef: 0.6931 - val_loss: 0.4933 - val_dice_coef: 0.9107 - val_jaccard_coef: 0.6864

Epoch 00003: val_los